In [1]:
# 安裝yahoo finance套件
!pip install yfinance

In [13]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間


# 爬取股票資訊的類別
class StockInfo():

    # 取得全部股票的股號、股名
    def stock_name(self):
      try:
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2', timeout=10)
        response.raise_for_status()  # 檢查 HTTP 回應狀態碼
      except requests.exceptions.HTTPError as http_err:
        print(f"HTTP 錯誤發生：{http_err}")
        return None
      except requests.exceptions.ConnectionError as conn_err:
          print(f"連線錯誤發生：{conn_err}")
          return None
      except requests.exceptions.Timeout as timeout_err:
          print(f"超時錯誤發生：{timeout_err}")
          return None
      except requests.exceptions.RequestException as req_err:
          print(f"請求錯誤發生：{req_err}")
          return None

      url_data = BeautifulSoup(response.text, 'html.parser')
      stock_company = url_data.find_all('tr')

      # 資料處理
      data = [
          (row.find_all('td')[0].text.split('\u3000')[0].strip(),
              row.find_all('td')[0].text.split('\u3000')[1],
              row.find_all('td')[4].text.strip())
          for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
      ]

      df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
      return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        return name_df.set_index('股號').loc[stock_id, '股名']

In [14]:
# 分析股票的類別
class StockAnalysis():

    def __init__(self, gemini_api_key):
        # 初始化 Gemini API 金鑰
        genai.configure(api_key=gemini_api_key)
        '''
        可以根據需求更換模型
            * gemini-1.5-flash
                - 輸入內容：音訊、圖片、影片和文字
                - 輸出：文字
                - 適合用途：在各種任務中提供快速且多功能的效能
            * gemini-1.5-flash-8b
                - 輸入內容：音訊、圖片、影片和文字
                - 輸出：文字
                - 適合用途：大量且較不智慧的工作
            * gemini-1.5-pro
                - 輸入內容：音訊、圖片、影片和文字
                - 輸出：文字
                - 適合用途：需要更多智慧功能的複雜推論工作
        '''
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        self.stock_info = StockInfo()  # 實例化 StockInfo 類別
        self.name_df = self.stock_info.stock_name()  # 取得台股資訊


    # 從 yfinance 取得兩周股價資料
    def stock_price(self, stock_id, days=15):
        stock_id += '.TW'

        end = dt.date.today() # 資料結束時間
        start = end - dt.timedelta(days=days) # 資料開始時間
        # 下載資料
        df = yf.download(stock_id, start=start)

        # 更換列名
        df.columns = ['開盤價', '最高價', '最低價',
                    '收盤價', '調整後收盤價', '成交量']

        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(),
            '收盤價': df['收盤價'].tolist(),
            '每日報酬': df['收盤價'].pct_change().tolist(),
            # '漲跌價差': df['調整後收盤價'].diff().tolist()
        }
        return data


    # 基本面資料
    def stock_fundamental(self, stock_id):
        stock_id += '.TW'
        stock = yf.Ticker(stock_id)
        # 營收成長率
        quarterly_revenue_growth = np.round(
            stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
            2
        )

        # 每季EPS
        quarterly_eps = np.round(
            stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
            2
        )

        # EPS季增率
        quarterly_eps_growth = np.round(
            stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
            2
        )

        # 轉換日期
        dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

        data = {
            '季日期': dates[:len(quarterly_revenue_growth)],  # 以最短的數據列表長度為准，確保數據對齊
            '營收成長率': quarterly_revenue_growth.tolist(),
            'EPS': quarterly_eps.tolist(),
            'EPS 季增率': quarterly_eps_growth.tolist()
        }

        return data


    # 新聞資料
    def stock_news(self, stock_name):
      data = []
      try:
        # 鉅亨網（Anue）的新聞 API，取得 Json 格式資料
        json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=6&page=1', timeout=10).json()
      except requests.exceptions.HTTPError as http_err:
          print(f"HTTP 錯誤發生：{http_err}")
          return data
      except requests.exceptions.ConnectionError as conn_err:
          print(f"連線錯誤發生：{conn_err}")
          return data
      except requests.exceptions.Timeout as timeout_err:
          print(f"超時錯誤發生：{timeout_err}")
          return data
      except requests.exceptions.RequestException as req_err:
          print(f"請求錯誤發生：{req_err}")
          return data


      # 依照格式擷取資料
      items=json_data['data']['items']
      for item in items:
        # 網址、標題和日期
        news_id = item['newsId']
        title = item['title']
        publish_at = item['publishAt']

        # 使用 UTC 時間格式
        utc_time = dt.datetime.utcfromtimestamp(publish_at)
        formatted_date = utc_time.strftime('%Y-%m-%d')

        try:
          # 前往網址擷取內容
          url = requests.get(f'https://news.cnyes.com/news/id/{news_id}', timeout=10).content
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP 錯誤發生：{http_err}")
            continue
        except requests.exceptions.ConnectionError as conn_err:
            print(f"連線錯誤發生：{conn_err}")
            continue
        except requests.exceptions.Timeout as timeout_err:
            print(f"超時錯誤發生：{timeout_err}")
            continue
        except requests.exceptions.RequestException as req_err:
            print(f"請求錯誤發生：{req_err}")
            continue

        soup = BeautifulSoup(url, 'html.parser')
        p_elements=soup .find_all('p')

        # 提取段落内容
        p=''
        for paragraph in p_elements[4:]:
            p += paragraph.get_text()
        data.append([stock_name, formatted_date ,title,p])
      return data


    # 取得 gemini 的回應
    def _get_reply(self, content_msg):
        response = self.model.generate_content(
            content_msg, # 傳送的訊息
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=1600,  # 限制回應的最大 Token 數為 1600
                temperature=1.0,  # 調整回應的隨機性（1.0 為適中的隨機性，範圍[0.0, 2.0]，值越高越有創意，值越低個準確）
            )
        )
        reply = response.text
        return reply

    def stock_gimini_sort(self, message):
        content_msg = f'''你現在是一位專業的股票分析師, 會根據各股的專業趨勢分析報告去評斷適不適合投資, 並給予0-100之間的評分。
            以 50 分為基準, 有任何正面消息可以加分如: 股價整體上升、法人買超、營收成長上升、新聞有正面消息；
            若有任何負面消息必須扣分如: 股價整體下降、法人賣超、營收成長下降、新聞有負面消息。
            最後請將所有股票依照評分排序出來。{str(message)}(使用繁體中文回應)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        content_msg = f'''你現在是一位專業的證券分析師, 你會針對各股的專業趨勢分析報告,
            選擇出最適合投資的一檔股票，即便目前都不適合投資也要一定要選出一檔，說明選擇它的理由。
            {str(message)}(使用繁體中文回應)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_analysis(self, stock_id):
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 取得股價資訊
        price_data = self.stock_price(stock_id)

        # 取得新聞資訊
        news_data = self.stock_news(stock_name)

        # 告訴 Gemini 現在他是什麼角色
        content_msg = f'''你現在是一位專業的證券分析師, 你會依據以下資料來進行分析並給出一份完整的分析報告:
            近期價格資訊: \n{price_data}

            你現在是一位專業的證券分析師，你會依據以下資料來進行分析並給出一份完整的分析報告:
            近期價格資訊: \n{price_data}'''

        stock_value_data = self.stock_fundamental(stock_id)
        content_msg += f'每季營收資訊: \n{stock_value_data}'

        content_msg += f'''近期新聞資訊: \n{news_data}
            請給我{stock_name}近期的趨勢報告，請以詳細、嚴謹及專業的角度撰寫此報告，並提及重要的數字(使用繁體中文回應)。
        '''
        reply = self._get_reply(content_msg)
        return reply


---

In [ ]:
# Gemini API Key
gemini_api_key = "AIz"#'輸入Gemini API Key'
# 建立 StockAnalysis 類別
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [16]:
reply = stock_analysis.stock_gimini_analysis('2330')
print(reply)

[*********************100%***********************]  1 of 1 completed


## 台積電近期趨勢報告 (2024年11月14日至2024年11月29日)

**報告日期:** 2024年11月30日

**分析師:**  [您的姓名/證券分析師編號]


**一、 價格走勢分析:**

本報告分析台積電在2024年11月14日至2024年11月29日期間的價格走勢。從提供的數據顯示，台積電股價呈現震盪下跌趨勢，期間最高價為1035元，最低價為992元，收盤價從1025元跌至994元，跌幅約為3.04%。

| 日期        | 收盤價 | 每日報酬 (%) |
|-------------|---------|-----------------|
| 2024-11-14  | 1025.0  |     NaN          |
| 2024-11-15  | 1030.0  |     0.49         |
| 2024-11-18  | 1020.0  |    -0.97         |
| 2024-11-19  | 1025.0  |     0.49         |
| 2024-11-20  | 1025.0  |     0.00         |
| 2024-11-21  | 1010.0  |    -1.46         |
| 2024-11-22  | 1035.0  |     2.47         |
| 2024-11-25  | 1030.0  |    -0.48         |
| 2024-11-26  | 1010.0  |    -1.94         |
| 2024-11-27  | 1000.0  |    -0.99         |
| 2024-11-28  | 992.0   |    -0.80         |
| 2024-11-29  | 994.0   |     0.20         |


**觀察重點:**  股價在1000元至1035元區間震盪，但整體趨勢偏弱，顯示市場信心有所減弱。尤其在11月21日至11月29日期間，股價持續下探，顯示賣壓較大。


**二、 營收與獲利分析:**

提供的季度營收數據顯示，台積電的營收成長率呈現波動，但整體來看，2024年前三季的營收成長率維持正成長，其中2024年第二季的

# 收集多檔股票的趨勢報告

In [10]:
# 建立股票清單
stock_list = ['2330', '8249', '5484', '8374', '2359','2377','2464','2467','3533','4562']

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

for stock in stock_list:
    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(stock_analysis.stock_gimini_analysis(stock_id=stock))

        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()

# 印出所有股票的趨勢報告
content

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'2377': '## 微星科技股份有限公司近期趨勢報告 (2024年11月)\n\n**報告日期:** 2024年11月30日\n\n**分析師:**  [您的姓名/證券分析師代號]\n\n\n**一、 執行摘要:**\n\n微星科技(2377-TW)近期股價呈現震盪走跌趨勢，在11月14日至11月29日期間，收盤價從170.5元跌至166元，跌幅約為2.64%。儘管每日報酬率波動不大，但整體趨勢偏弱。此波股價下跌可能與整體大盤修正、AI伺服器相關個股短期獲利了結，以及市場對微星在AI伺服器領域布局進度的觀望情緒有關。然而，近期多則新聞顯示微星積極佈局AI伺服器市場，並獲得外資調升目標價，暗示其長期發展潛力仍然值得關注。  本報告將深入分析微星近期的價格走勢、財務狀況以及相關新聞資訊，以評估其未來發展趨勢。\n\n\n**二、 近期價格走勢分析:**\n\n下表顯示微星科技在2024年11月14日至29日期間的收盤價及每日報酬率：\n\n| 日期        | 收盤價 | 每日報酬率 |\n|-------------|---------|-------------|\n| 2024-11-14  | 170.5   | NaN          |\n| 2024-11-15  | 167.0   | -2.05%      |\n| 2024-11-18  | 165.5   | -0.90%      |\n| 2024-11-19  | 168.5   | 1.81%       |\n| 2024-11-20  | 171.5   | 1.78%       |\n| 2024-11-21  | 169.0   | -1.46%      |\n| 2024-11-22  | 172.5   | 2.07%       |\n| 2024-11-25  | 173.0   | 0.29%       |\n| 2024-11-26  | 172.0   | -0.58%      |\n| 2024-11-27  | 170.0   | -1.16%      |\n| 2024-11-28  | 168.0   | -1.18%      |\n| 2024-11-29  | 166.0   | -1.19%      |\n\n\n從數據觀察，

# 推薦一檔股票

In [11]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

身為一位專業的證券分析師，經過對提供的資料分析，我認為目前最適合投資的股票是**嘉澤(3533-TW)**，即便目前市場存在不確定性。

**理由如下:**

儘管近期嘉澤股價呈現震盪，但其基本面相對穩健，展現出比其他股票更強的抗跌性及成長潛力：

1. **強勁的營收成長:**  10月份營收年增率高達35.51%，顯示公司營運持續擴張，這遠優於其他幾檔股票提供的營收數據。持續高增長的營收是支撐股價的重要因素。

2. **相對穩定的獲利能力:** 雖然嘉澤的EPS季增率在不同季度有所波動，但整體而言，其EPS仍維持在相對較高的水平。與其他幾檔公司相比，嘉澤的獲利能力更為穩定，風險相對較低。  雖然第三季EPS季增率為負，但整體營收表現強勁，值得持續觀察。

3. **Factset樂觀預期:** 雖然Factset的EPS預估存在不確定性，但預估值仍相對樂觀，暗示市場對嘉澤的長期發展抱持正面看法，這與其他公司缺乏正面預期或預期下修形成對比。

4. **相對較低的股價:**  目前嘉澤股價相對低點，這提供了一個較低的進場成本，降低投資風險。  相對其他股票，其股價下跌幅度也較小。

5. **積極的投資策略:**  新聞中提及嘉澤購置不動產，顯示公司對未來發展充滿信心，並積極投資擴展業務。

**風險考量:**

當然，投資嘉澤也存在一定的風險，例如市場整體波動、產業競爭以及Factset預估的不確定性等。  因此，建議投資者採取分批進場策略，降低投資風險，並持續密切關注市場變化和公司財報。


**其他股票的評估:**

* **微星(2377):**  雖然積極佈局AI伺服器市場，具備長期發展潛力，但近期股價震盪下跌，且缺乏完整的財務數據，短期內存在較大的不確定性，風險較高。

* **盟立(2464):** 10月份營收增長亮眼，但EPS增長趨緩，且股價波動較大，缺乏持續性上漲趨勢，風險亦高。

* **志聖(2467):** 10月份營收增長顯著，但隨後股價持續下跌，財務數據顯示營運及獲利能力波動性大，風險較高。

* **穎漢(4562):**  營收表現不佳，EPS為負值，股價波動劇烈，風險過高，不建議投資。


**結論:**

綜上所述，在現有資訊下，嘉澤(3533-TW) 雖然股價近期震盪，但其基本面相對穩健，成長動能強勁，且風險相對可控，因此

# 推薦股票的評分排序

In [12]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

身為專業股票分析師，我根據提供的報告，對各支股票進行評分 (0-100)：

**評分標準:**

* **股價:** 上漲加分，下跌扣分，幅度影響分數多寡。
* **法人動向:** 買超加分，賣超扣分。
* **營收:** 成長上升加分，下降扣分，幅度影響分數多寡。
* **新聞:** 正面消息加分，負面消息扣分。
* **其他基本面:**  例如EPS、獲利能力等，也會納入考量。


**評分結果:**

* **2377 微星科技:** 股價小幅下跌，但有外資調升目標價的正面消息，以及積極佈局AI伺服器市場的利多，抵銷部分股價下跌的影響。 營收及EPS數據不足，難以給予高分。評分：**60**

* **2464 盟立:** 10月營收年增20.42%，為正面指標，但EPS增長趨緩，且股價波動較大。黃仁勳對機器人產業的正面評價也為其加分。  整體而言，基本面表現尚可，但股價波動風險較高。評分：**55**

* **2467 志聖:** 10月營收年增37.02%，但股價持續下跌，且第三季營收負成長，EPS也下滑。新聞多為負面訊息，顯示市場觀望情緒濃厚。評分：**40**

* **3533 嘉澤:** 10月營收年增35.51%，為極為正面的指標，雖然股價有震盪，但整體仍維持在相對高點。Factset預估EPS也相對樂觀，但需謹慎看待預估的準確性。評分：**75**

* **4562 穎漢:** 營收表現不佳，EPS為負值，股價雖一度上漲，但最終下跌，且新聞多為短期股價波動的報導，缺乏公司基本面改善的資訊。評分：**30**


**股票評分排序 (由高到低):**

1. **3533 嘉澤 (75)**
2. **2377 微星科技 (60)**
3. **2464 盟立 (55)**
4. **2467 志聖 (40)**
5. **4562 穎漢 (30)**


**免責聲明:** 以上評分及分析僅供參考，不構成任何投資建議。投資人應自行判斷，並承擔所有投資風險。  本分析基於提供的有限資訊，若能提供更完整的數據，分析結果可能會有調整。

